### download the rolling hindcasts for all GCMs from the Copernicus Climate Datastore

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser

In [34]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [35]:
import numpy as np

In [36]:
HOME = pathlib.Path.home()

In [37]:
CWD = pathlib.Path.cwd()

In [38]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [39]:
sys.path.append('../..')

In [40]:
from ICU_Water_Watch import C3S, domains

### show the predefined domains 

In [41]:
domains.domains

{'Tropical_Pacific': [140, 220, -25, 25],
 'SW_Pacific': [172.5, 190, -22.5, -12],
 'Fiji': [175, 183, -21, -15],
 'NZ': [161, 181, -50, -30],
 'Pacific': [140, 240, -50, 25],
 'C3S_download': [100, 240, -50, 30],
 'Water_Watch': [120, 240, -35, 25]}

### provider (always CDS for now)

In [42]:
provider = 'CDS'

### variable name

In [43]:
var_name = 'tprate'

### mapping GCMs to systems 

#### The mapping between the GCM name and the latest system version available for the forecasts needs to be checked [on the CDS website](https://cds.climate.copernicus.eu/)

In [44]:
from collections import OrderedDict as OD

In [45]:
dict_systems = OD(
                ECMWF=5, # SEAS5, hindcast complete 
                UKMO=600, # note: only for system 14 is hindcast complete
                METEO_FRANCE=8, # note: only for system 7 is hindcast complete
                DWD=21, # GCFS2.1, hindcast complete 
                CMCC=35, # SPSv3.5, hindcast complete 
                NCEP=2,# CFSv2, hindcast complete 
                JMA=2, # CPS2, hindcast complete
                ECCC_GEM_NEMO=1, # GEM-NEMO, hindcast incomplete
                ECCC_CanCM4i=2, # CanCM4i, hindcast incomplete
                )

In [46]:
dict_systems.keys()

odict_keys(['ECMWF', 'UKMO', 'METEO_FRANCE', 'DWD', 'CMCC', 'NCEP', 'JMA', 'ECCC_GEM_NEMO', 'ECCC_CanCM4i'])

### lag in months (if need to download older forecasts)

In [47]:
lag = 1

### get today's date 

In [48]:
date = dateparser.parse('today')

### apply lag 

In [49]:
date = date - relativedelta(months=lag)

In [50]:
date

datetime.datetime(2021, 10, 17, 14, 22, 27, 246244)

### generate the dates for the download of the rolling hindcasts 

In [51]:
dates = [datetime(y, date.month, 1) for y in range(1993, 2016 + 1)]

In [52]:
dates

[datetime.datetime(1993, 10, 1, 0, 0),
 datetime.datetime(1994, 10, 1, 0, 0),
 datetime.datetime(1995, 10, 1, 0, 0),
 datetime.datetime(1996, 10, 1, 0, 0),
 datetime.datetime(1997, 10, 1, 0, 0),
 datetime.datetime(1998, 10, 1, 0, 0),
 datetime.datetime(1999, 10, 1, 0, 0),
 datetime.datetime(2000, 10, 1, 0, 0),
 datetime.datetime(2001, 10, 1, 0, 0),
 datetime.datetime(2002, 10, 1, 0, 0),
 datetime.datetime(2003, 10, 1, 0, 0),
 datetime.datetime(2004, 10, 1, 0, 0),
 datetime.datetime(2005, 10, 1, 0, 0),
 datetime.datetime(2006, 10, 1, 0, 0),
 datetime.datetime(2007, 10, 1, 0, 0),
 datetime.datetime(2008, 10, 1, 0, 0),
 datetime.datetime(2009, 10, 1, 0, 0),
 datetime.datetime(2010, 10, 1, 0, 0),
 datetime.datetime(2011, 10, 1, 0, 0),
 datetime.datetime(2012, 10, 1, 0, 0),
 datetime.datetime(2013, 10, 1, 0, 0),
 datetime.datetime(2014, 10, 1, 0, 0),
 datetime.datetime(2015, 10, 1, 0, 0),
 datetime.datetime(2016, 10, 1, 0, 0)]

### path to download the rolling hindcasts 

In [53]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/{provider}/operational/hindcasts')

In [54]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### Now loops over the dates, creates a dictionnary containing the keywords arguments, and pass to dask for parallel download 

In [55]:
list_GCMs = ['ECMWF', 'UKMO', 'METEO_FRANCE', 'DWD', 'CMCC', 'NCEP', 'JMA', 'ECCC_CanCM4i', 'ECCC_GEM_NEMO']

In [56]:
gcm_path

PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts')

In [57]:
for date in dates: 
    
    downloaded_files = []

    # for GCM in dict_systems.keys():
    for GCM in list_GCMs[-2:]:

        if 'ECCC' in GCM: 
            
            GCM_API_name = GCM.split('_')[0]
            GCM_path = gcm_path.joinpath(GCM).joinpath(var_name.upper())
            
            print(f"processing ECCC, path should be {str(GCM_path)}")
        
            args = dict(GCM=GCM_API_name,
            system=dict_systems[GCM],
            varname=var_name,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=GCM_path,
            domain=domains.domains['C3S_download'],
            file_format='netcdf',
            level='surface',
            max_retry=3)
        
        else: 

            args = dict(GCM=GCM,
            system=dict_systems[GCM],
            varname=var_name,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
            domain=domains.domains['C3S_download'],
            file_format='netcdf',
            level='surface',
            max_retry=3)
            

        downloaded_files.append(delayed(C3S.download)(**args))

    with ProgressBar():
        downloaded_files = compute(downloaded_files)

processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECCC_CanCM4i/TPRATE
processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECCC_GEM_NEMO/TPRATE
[                                        ] | 0% Completed |  0.0s
/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECCC_GEM_NEMO/TPRATE/ensemble_seas_forecasts_tprate_from_1993_10_ECCC.netcdf exists already on disk, skipping download and returning path


attempting to download tprate for GCM ECCC, for level surface, year 1993, month 10, in netcdf, attempt 1 of 3

[####################                    ] | 50% Completed |  1.8s

2021-11-17 14:22:34,285 INFO Welcome to the CDS
2021-11-17 14:22:34,286 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels


[####################                    ] | 50% Completed |  2.2s

2021-11-17 14:22:34,685 INFO Request is queued


[####################                    ] | 50% Completed | 25.1s

...

[####################                    ] | 50% Completed |  1min 33.3s

 99%|█████████▉| 1.08M/1.09M [00:10<00:00, 103kB/s] 

[####################                    ] | 50% Completed |  1min 33.4s

2021-11-17 15:06:13,752 INFO Download rate 91.8K/s 


[####################                    ] | 50% Completed |  1min 33.7s
/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECCC_CanCM4i/TPRATE/ensemble_seas_forecasts_tprate_from_2016_10_ECCC.netcdf downloaded OK

[########################################] | 100% Completed |  1min 33.8s
